In [121]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat


data_all_participants = []
labels = []

for participant_id in range(1, 6):  
    filename = f"../Sub{participant_id}_1_multitarget.mat"
    f = loadmat(filename)
    data = f['Data'][0][0][7]
    label = f['Data'][0][0][8][0] 

    filename1 = f"../Sub{participant_id}_2_multitarget.mat"
    f1 = loadmat(filename1)
    data1 = f1['Data'][0][0][7]
    label1 = f1['Data'][0][0][8][0] 
    
    participant_data = np.concatenate((data,data1), axis=1)
    labels_data = np.concatenate((label,label1))
    
    data_all_participants.append(participant_data)
    labels.append(labels_data)

data_all_participants = np.array(data_all_participants)
labels = np.array(labels)
labels = np.concatenate(labels)

print(labels.shape)  
print(data_all_participants.shape)  


(100,)
(5, 8192, 20)


In [122]:
# inicialmente criamos o objeto info
import mne

n_channels = 1
sfreq = 512
ch_names = ['Oz']
ch_types = ['eeg']
info = mne.create_info(ch_names, sfreq=sfreq, ch_types=ch_types)
info

<Info | 7 non-empty values
 bads: []
 ch_names: Oz
 chs: 1 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 256.0 Hz
 meas_date: unspecified
 nchan: 1
 projs: []
 sfreq: 512.0 Hz
>

In [124]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Shape dos dados: (n_epochs, n_channels, n_times) => (100, 1, 8192)
data_correct = data_all_participants.transpose(0,2,1).reshape(-1,1,8192)
print(data_correct.shape)

labels = labels.astype(str)

# Continuar com a codificação dos labels
le = LabelEncoder() #fazer um zip para o event_dict ?
encoded_labels = le.fit_transform(labels)

# Criar a matriz de eventos
events = np.column_stack((
    np.array(range(len(labels))),
    np.zeros(labels.shape[0], dtype=int),
    encoded_labels
))

# organizar os targets de acordo com todos os diferentes valores de labels
event_dict = {'6.0':0, '6.5': 1, '7.0':2, '7.5':3, '8.2':4, '9.3': 5}
mne_data = mne.EpochsArray(data_correct, info, events, event_id=event_dict)
mne_data.save('ssvep-epo.fif', overwrite=True)

(100, 1, 8192)
Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated


Overwriting existing file.
Overwriting existing file.
